In [1]:
import numpy as np
from scipy import special as sp
from scipy.optimize import fsolve
import csv
import copy
from constitutive import *
from domegoveqn import *

#need matlab to check
import matlab
import matlab.engine


In [2]:
eng = matlab.engine.start_matlab()

In [3]:
eng.run('../domeconduit/Code/RunModelScripts/AddPaths.m', nargout= 0)
eng.addpath('../domeconduit/Code/RunModelScripts/', nargout= 0)
eng.addpath ('../MassMatrix/', nargout= 0 )

In [4]:
# Model parameters
m = {}


# temperature
Tvec = np.array((950, 900, 850, 800, 750))
m['Ti'] = 3
m['T'] = np.float(Tvec[m['Ti']-1] + 273.15)  # temperature [K]

# geometry, all in meters
m['R'] = 50
m['conduit_length'] = 4e3

m['g'] = 9.81    # gravitational acceleration, m/s2

# conduit base boundary conditions
m['p_ch'] = 105428000.
m['p_top']= 1e5
m['chi_ch'] = {'h2o': 5.0, 'co2': 2000.0}

# densities
m['rho_s']  = 2600.
m['rho_l']  = 2200.
m['rho_hd'] = 2200.
m['rho_cd'] = 2200.

# gas constants
m['Rw'] = 461.5         # J/(kg K)
m['Rc'] = 188.9         # J/(kg K)
m['B']  = 18.02/44.01;  # molar mass ratio water/co2
m['eta_g'] = 1e-4       # gas viscosity, Pa s

# permeability 
m['kc']     = 1e-13    # magma permeability constant, m2
m['phi_gc'] = 0.0     # percolation threshold

# wall rock permeability properties
klw        = {}       
klw['top'] = 1e-14    # wall rock permeability at top, m2
klw['mi']  = 3.2      # wall rock permeability exponent in Manning & Ingebritsen 1999
klw['L']   = m['R']   # length scale for wall rock gas percolation
m['klw']   = klw

# hydrostatic pressure
m['phydro'] = {'slope':9810}

# friction
fr = {}
fr['v_ref'] = 1e-5    # reference vel for friction, m/s
fr['f0']    = 0.1     # nominal friction coeff
fr['a']     = 0.01    # velocity dependence of friction
fr['A']     = 2*fr['v_ref']*np.exp(-fr['f0']/fr['a'])   # prefactor in MBE
m['fr']     = fr

# normal stress
m['sig']      = {'slope':16677., 'offset':1000000.}

# plug definition
m['newtonian'] = {'vvfrac_thr':0.5}



# initialise solid fraction curves
fname = 'chiscurves/chis_ti' + str(m['Ti']) + '.csv'
with open(fname, newline='') as f:
    reader = np.array(list(csv.reader(f)),dtype=np.float)

pp = {}
pp['breaks'] = np.array(reader[:,0]).reshape(len(reader),1)
pp['coefs'] = reader[:-1,1:]
m['pf'] = pp


print(m['pf']['coefs'])

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 6.16015608e-05 -2.15651954e-03  0.00000000e+00  1.00000000e+00]
 [-7.55844999e-04  2.47219765e-03 -1.45701025e-02  6.40432922e-01]
 [ 9.82827090e-05  4.29780870e-04 -1.20319017e-02  6.29646551e-01]
 [ 1.35745200e-04  1.81917244e-04 -9.54498540e-03  6.10032362e-01]
 [-1.76691744e-04  9.53765493e-04 -6.45645994e-03  5.90411746e-01]
 [-2.96973259e-05  3.20919101e-04 -4.75153410e-03  5.81542334e-01]
 [-1.61465206e-06  1.08374807e-04 -3.84473338e-03  5.73354908e-01]
 [ 9.14471381e-07  3.52348682e-05 -3.05572896e-03  5.59615057e-01]
 [-1.26441348e-06  4.42273444e-05 -2.22059052e-03  5.36028642e-01]
 [-2.21728020e-07  1.24713344e-05 -1.70703056e-03  5.12874014e-01]
 [-1.34878356e-08  9.22760289e-08 -1.56663012e-03  4.65964097e-01]
 [ 2.31034495e-08 -3.39146578e-06 -1.70426866e-03  3.68282024e-01]]


### the next few cells are just to check that the python code for constitutive relations is the same as the matlab functions

In [5]:
# check phase fraction derivatives
p = mat_to_py(np.array([100e6,75e6,50e6,25e6]))
phi_g = mat_to_py(np.array([0.1,0.2,0.3,0.3]))
mh = mat_to_py(np.array([0.9,0.92,0.95,0.98]))

h2o, co2, c, phi, rho = density (p, phi_g, mh, m)

pyout = {}
pyout['h2o_d'] = h2o['dissolved']
pyout['co2_d'] = co2['dissolved']
pyout['c1']    = c['c1']
pyout['c2']    = c['c2']
pyout['chi_s'] = phi['chi_s']['chi_s']
pyout['phi_s'] = phi['s']
pyout['phi_l'] = phi['l']
pyout['rho']   = rho['mix']
pyout['dh2o_d']= {'p':h2o['dp']         , 'mh':h2o['dmh']}
pyout['dco2_d']= {'p':co2['dp']         , 'mh':co2['dmh']}
pyout['dc1']   = {'p':c['dc1']['dp']    , 'mh':c['dc1']['dmh']}
pyout['dc2']   = {'p':c['dc2']['dp']    , 'mh':c['dc2']['dmh']}
pyout['dphis'] = {'p':phi['dphis']['dp'], 'mh':phi['dphis']['dmh'], 'phi_g':phi['dphis']['dphi_g']}
pyout['dphil'] = {'p':phi['dphil']['dp'], 'mh':phi['dphil']['dmh'], 'phi_g':phi['dphil']['dphi_g']}


In [6]:

pm = matlab.double(p.tolist())
phigm = matlab.double(phi_g.tolist())
mhm = matlab.double(mh.tolist())

mtmp = copy.deepcopy(m)
mtmp.pop('pf', None)

pp = {'breaks': matlab.double(m['pf']['breaks'].tolist()), 'coefs': matlab.double(m['pf']['coefs'].tolist())}
mtmp['pf'] = {'pp':pp}

mtmp.pop('fr', None)

Chi_hd, Chi_cd, c1, c2, chi_s, phi_s, phi_l, rho, dChihd, dChicd, dc1, dc2, dphis, dphil = eng.phasefracderivs(pm, phigm, mhm, mtmp, nargout=14)

mlout = {}
mlout['h2o_d'] = np.array(Chi_hd)
mlout['co2_d'] = np.array(Chi_cd)
mlout['c1']    = np.array(c1)
mlout['c2']    = np.array(c2)
mlout['chi_s'] = np.array(chi_s)
mlout['phi_s'] = np.array(phi_s)
mlout['phi_l'] = np.array(phi_l)
mlout['rho']   = np.array(rho)
mlout['dh2o_d']= np.array(dChihd)
mlout['dco2_d']= np.array(dChicd)
mlout['dc1']   = np.array(dc1)
mlout['dc2']   = np.array(dc2)
mlout['dphis'] = np.array(dphis)
mlout['dphil'] = np.array(dphil)


In [7]:
# prints out and compares all the variables between the matlab and python codes


print('python code: rho = ', pyout['rho'].T)
print('matlab code: rho = ', mlout['rho'].T, '\n')

print('python code: c1 = ', pyout['c1'].T)
print('matlab code: c1 = ', mlout['c1'].T, '\n')

print('python code: c2 = ', pyout['c2'].T)
print('matlab code: c2 = ', mlout['c2'].T, '\n')

print('python code: dh2o_d/dp = ', pyout['dh2o_d']['p'].T)
print('matlab code: dh2o_d/dp = ', mlout['dh2o_d'][:,0].T, '\n')

print('python code: dh2o_d/dmh = ', pyout['dh2o_d']['mh'].T)
print('matlab code: dh2o_d/dmh = ', mlout['dh2o_d'][:,1].T, '\n')

print('python code: dco2_d/dp = ', pyout['dco2_d']['p'].T)
print('matlab code: dco2_d/dp = ', mlout['dco2_d'][:,0].T, '\n')

print('python code: dco2_d/dmh = ', pyout['dco2_d']['mh'].T)
print('matlab code: dco2_d/dmh = ', mlout['dco2_d'][:,1].T, '\n')


print('python code: dc1/dp = ', pyout['dc1']['p'].T)
print('matlab code: dc1/dp = ', mlout['dc1'][:,0].T, '\n')

print('python code: dc1/dmh = ', pyout['dc1']['mh'].T)
print('matlab code: dc1/dmh = ', mlout['dc1'][:,1].T, '\n')


print('python code: dc2/dp = ', pyout['dc2']['p'].T)
print('matlab code: dc2/dp = ', mlout['dc2'][:,0].T, '\n')

print('python code: dc2/dmh = ', pyout['dc2']['mh'].T)
print('matlab code: dc2/dmh = ', mlout['dc2'][:,1].T, '\n')



print('python code: dphis/dp = ', pyout['dphis']['p'].T)
print('matlab code: dphis/dp = ', mlout['dphis'][:,0].T, '\n')

print('python code: dphis/dphig = ', pyout['dphis']['phi_g'].T)
print('matlab code: dphis/dphig = ', mlout['dphis'][:,1].T, '\n')

print('python code: dphis/dmh = ', pyout['dphis']['mh'].T)
print('matlab code: dphis/dmh = ', mlout['dphis'][:,2].T, '\n')


print('python code: dphil/dp = ', pyout['dphil']['p'].T)
print('matlab code: dphil/dp = ', mlout['dphil'][:,0].T, '\n')

print('python code: dphil/dphig = ', pyout['dphil']['phi_g'].T)
print('matlab code: dphil/dphig = ', mlout['dphil'][:,1].T, '\n')

print('python code: dphil/dmh = ', pyout['dphil']['mh'].T)
print('matlab code: dphil/dmh = ', mlout['dphil'][:,2].T, '\n')

python code: rho =  [[2159.0937166  1944.16366888 1722.2065867  1834.24726395]]
matlab code: rho =  [[2159.0937166  1944.16366888 1722.2065867  1834.24726395]] 

python code: c1 =  [[1.03796513 1.03247169 1.02613116 1.01792041]]
matlab code: c1 =  [[1.03796513 1.03247169 1.02613116 1.01792041]] 

python code: c2 =  [[0.9634235  0.96854956 0.97453429 0.98239507]]
matlab code: c2 =  [[0.9634235  0.96854956 0.97453429 0.98239507]] 

python code: dh2o_d/dp =  [[2.08740937e-10 2.38465594e-10 2.87041498e-10 3.88404630e-10]]
matlab code: dh2o_d/dp =  [2.08740937e-10 2.38465594e-10 2.87041498e-10 3.88404630e-10] 

python code: dh2o_d/dmh =  [[0.02569807 0.02094031 0.01584116 0.01013028]]
matlab code: dh2o_d/dmh =  [0.02569807 0.02094031 0.01584116 0.01013028] 

python code: dco2_d/dp =  [[6.31131807e-13 4.99306140e-13 3.12690527e-13 1.25823972e-13]]
matlab code: dco2_d/dp =  [6.31131807e-13 4.99306140e-13 3.12690527e-13 1.25823972e-13] 

python code: dco2_d/dmh =  [[-0.00062137 -0.00046553 -0.

In [8]:
# check viscosity

h2o_d = 5.
phi_s_eta = 0.5
gdot = 1.0e-5

eta1  = viscosity(h2o_d, phi_s_eta, gdot, m)
eta2, eta_m2, eta_phi2 = eng.tdcFV('calc_eta', h2o_d*0.01, phi_s_eta, m['T'], 2, gdot, nargout=3)

print('python code: ', eta1['mix'], eta1['melt'], eta1['xtal'])
print('matlab code: ', eta2, eta_m2, eta_phi2)



python code:  233815.67501426488 5832.851929387548 40.08599529781234
matlab code:  233815.67501426488 5832.851929387548 40.08599529781234


In [9]:
# check solid mass fraction

p = mat_to_py(np.array([100e6,50e6]))
chi_s = solid_mass_frac(p*1e-6, m['pf'])
chi_s['dchisdp'] = solid_mass_frac_deriv(chi_s)


print('python code: chi_s = ', chi_s['chi_s'].T)
print('matlab code: chi_s = ', 0.477586596754590,0.581054531506694)

print('python code: dchis/dp = 1e-8 x', 1e8*chi_s['dchisdp'].T)
print('matlab code: dchis/dp = 1e-8 x', -0.148494868567476,  -0.468613491456686)


python code: chi_s =  [[0.4775866  0.58105453]]
matlab code: chi_s =  0.47758659675459 0.581054531506694
python code: dchis/dp = 1e-8 x [[-0.14849487 -0.46861349]]
matlab code: dchis/dp = 1e-8 x -0.148494868567476 -0.468613491456686


In [10]:
# check permeability
degas =  1
p     =  1e7*mat_to_py(np.array([9.126853500379374,   7.746970469494080,   6.427370851459487,   5.174760554871329]))
phi_g =      mat_to_py(np.array([0.104233779536040,   0.144496818364394,   0.189868604799967,   0.238369127138891]))
dpdz  = -1e4*mat_to_py(np.array([2.110369216779560,   2.026756139387187,   1.930246732696368,   1.828099332132594]))
z     = -1e3*mat_to_py(np.array([3.340000000000000,   2.673333333333333,   2.006666666666667,   1.340000000000000]))

k = permeability(phi_g, degas, z, m)
gvel = gasvels(p, phi_g, dpdz, degas, z, m)

print('python code: kvert = 1e-14 x', k['vert'].T*1e14)
print('matlab code: kvert = 1e-14 x [ 0.011324667428581   0.030169968293476   0.068447797384461   0.135440958492205]\n')

print('python code: klat = 1e-14 x', k['lat'].T*1e14)
print('matlab code: klat = 1e-14 x [ 0.014735594318271   0.035458845924748   0.083690262157321   0.201319910330897]\n')

print('python code: (vg-v) = 1e-7 x', gvel['vg']['vel'].T*1e7)
print('matlab code: (vg-v) = 1e-7 x [0.238992295315426   0.611471684639190   1.321211372616185   2.4759952576299889]\n')

print('python code: ug = 1e-6 x', gvel['ug'].T*1e6)
print('matlab code: ug = 1e-6 x [0.862078463762972   1.817063904698754   3.731607228737932   7.771392559641819]\n')

python code: kvert = 1e-14 x [[0.01132467 0.03016997 0.0684478  0.13544096]]
matlab code: kvert = 1e-14 x [ 0.011324667428581   0.030169968293476   0.068447797384461   0.135440958492205]

python code: klat = 1e-14 x [[0.01473559 0.03545885 0.08369026 0.20131991]]
matlab code: klat = 1e-14 x [ 0.014735594318271   0.035458845924748   0.083690262157321   0.201319910330897]

python code: (vg-v) = 1e-7 x [[0.2389923  0.61147168 1.32121137 2.47599526]]
matlab code: (vg-v) = 1e-7 x [0.238992295315426   0.611471684639190   1.321211372616185   2.4759952576299889]

python code: ug = 1e-6 x [[0.86207846 1.8170639  3.73160723 7.77139256]]
matlab code: ug = 1e-6 x [0.862078463762972   1.817063904698754   3.731607228737932   7.771392559641819]



### now assemble equations for steady-state system and check against ss matlab solution

In [28]:
# run steady-state matlab code

o = eng.tdcFV('setdef', nargout=1)
o['phi_gc'] = 0
ss, opts, ssflag = eng.tdcFV('run_ssc', o, nargout=3);
ss = eng.smf_rad_dz('add_fields',ss)




In [43]:
mtmp = reformat_params(ss['m'])
print(mtmp['fr'])

# check chamber volatile boundary conditions
mh_ch, phi_g_ch = chambervolatiles(mtmp)
print(m['p_ch'], mh_ch, phi_g_ch)
print(ss['m']['p_ch'], ss['m']['mw_ch'], ss['m']['phi_g_ch'])

{'v_ref': 1e-05, 'f0': 0.1, 'a': 0.01, 'A': 9.079985952496972e-10}
105428000.0 [0.93457391] [[0.07042058]]
105428000.0 0.9345739121101044 0.07042058317629579


In [33]:
# collect variables from ss matlab code for comparison with py code

# get d mass matrix from symbolic manipulator results to check elements of J from ssc
dMsl, dMh, dMc = eng.pychecks(ss,nargout=3)
dMsl = mat_to_py(dMsl)
dMh = mat_to_py(dMh)
dMc = mat_to_py(dMc)

Qh = 2*mat_to_py(ss['rho_g'])*mat_to_py(ss['phi_g'])*mat_to_py(ss['u_lat'])/m['R']/(1+mat_to_py(ss['Gamma']))
Qc = Qh*mat_to_py(ss['Gamma'])

phi_s = mat_to_py(ss['phi_s'])
phi_l = mat_to_py(ss['phi_l'])
vv = mat_to_py(ss['vv'])
vf = mat_to_py(ss['vf'])
eta = mat_to_py(ss['eta'])
chihd = mat_to_py(ss['Chi_hd'])
chicd = mat_to_py(ss['Chi_cd'])
tauR = mat_to_py(ss['tauR'])
ssrho = mat_to_py(ss['rho'])
plug = mat_to_py(ss['state'])

print(ss['m']['p_ch'], ss['m']['chi_ch'])


105428000.0 {'total': {'h2o': 5.0, 'co2': 2000.0}}


In [45]:
# initialise variables for use in py code

Nz    = np.int(opts['Nz'])
ind   = np.arange(0,Nz,100)
tmp   = -mat_to_py(ss['z'])
z     = tmp[ind]

p     = mat_to_py(ss['p'])
v     = mat_to_py(ss['v'])
phi_g = mat_to_py(ss['phi_g'])
mh    = mat_to_py(ss['mw'])

ddz   = mat_to_py(ss['yp'])

var          = {}
var['p']     = p[ind]
var['v']     = v[ind]
var['phi_g'] = phi_g[ind]
var['mh']    = mh[ind]

p = {}
v = {}
phi_g = {}
mh = {}

dpdz = mat_to_py(ss['pp'])
p['z']     = -(dpdz[ind]) #-mat_to_py(ddz[ind,0])
v['z']     = -mat_to_py(ddz[ind,1])
phi_g['z'] = -mat_to_py(ddz[ind,2])
mh['z']    = -mat_to_py(ddz[ind,3])

grd = {'p': p, 'v':v, 'phi_g':phi_g, 'mh':mh}
print(var['v'])

nu = np.squeeze(np.array([var['p'], var['v'],var['phi_g'],var['mh'],grd['p']['z'], grd['v']['z'] ,grd['phi_g']['z'],grd['mh']['z']] ).T)

# indices of variables
mtmp['vi'] = {'p':0, 'v':1, 'phi_g':2, 'mh':3, 'pz':4, 'vz':5, 'phi_gz':6, 'mhz':7}

print(grd)


[[0.00091651]
 [0.00094601]
 [0.00098479]
 [0.00103332]
 [0.00108957]
 [0.0011804 ]
 [0.00114392]]
{'p': {'z': array([[-21777.90523538],
       [-21096.09241721],
       [-20258.45573503],
       [-19292.2955338 ],
       [-18271.06157059],
       [-88063.95891573],
       [-22866.62015994]])}, 'v': {'z': array([[ 3.79248970e-08],
       [ 5.08775029e-08],
       [ 6.56975612e-08],
       [ 8.01009640e-08],
       [ 8.65094983e-08],
       [-3.46859894e-07],
       [-3.63370154e-18]])}, 'phi_g': {'z': array([[4.67328882e-05],
       [5.58757962e-05],
       [6.49270287e-05],
       [7.16720634e-05],
       [7.27367655e-05],
       [9.53018550e-05],
       [1.70145564e-15]])}, 'mh': {'z': array([[3.25173176e-05],
       [1.87916385e-05],
       [1.24561601e-05],
       [9.34415150e-06],
       [8.15369640e-06],
       [2.46132004e-14],
       [7.44779238e-16]])}}


In [46]:
# run ssc code and print values for checking

J, Q, iplug, mbe, phi, h2o, co2, rho = ssc(z, var, grd, mtmp)

im = np.arange(0,301,50)

print('Check values (order: matlab; python)')

print('solid phase fraction')
print(phi_s[ind].T)
print(phi['s'].T)
print('\n')

print('liquid phase fraction')
print(phi_l[ind].T)
print(phi['l'].T)
print('\n')

print('dissolved h2o')
print(chihd[ind].T)
print(h2o['dissolved'].T)
print('\n')

print('dissolved co2')
print(chicd[ind].T)
print(co2['dissolved'].T)
print('\n')

print('viscosity')
print(eta[ind].T)
print(mbe['eta'].T)
print('\n')

print('density')
print(ssrho[ind].T)
print(rho['mix'].T)
print('\n')

print('conduit wall shear stress')
print(tauR[ind].T)
print(mbe['tau_R'].T)
print('\n')

print('viscous velocity')
print(vv[ind].T)
print(mbe['vv'].T)
print('\n')

print('frictional velocity')
print(vf[ind].T)
print(mbe['vfr'].T)
print('\n')

print('total velocity')
print(var['v'].T)
print(mbe['vv'].T + mbe['vfr'].T)
print('\n')


print('plug?')
print(plug[ind,1].T)
print(iplug)
print('Do not expect water and co2 continuity to match when iplug == 1.\n')

print('solids and liquids continuity')
print('v x d sl/dp')
print(var['v'].reshape(len(ind))*dMsl[im,0])
print(J[:,m['vi']['p'],m['vi']['pz']])
print('\n')

print('v x d sl/dphig')
print(var['v'].reshape(len(ind))*dMsl[im,2])
print(J[:,m['vi']['p'],m['vi']['phi_gz']])
print('\n')

print('v x d sl/dmh ')
print(var['v'].reshape(len(ind))*dMsl[im,3])
print(J[:,m['vi']['p'],m['vi']['mhz']])
print('\n')
print('\n')

# NB If you want to compare the bottom two sets, you need to comment out the terms in ssc starting with vgrshp, 
# since the mass matrix does not include that
print('water continuity')
print('v x d h/dp ')
print(var['v'].reshape(len(ind))*dMh[im,0])
print(J[:,m['vi']['phi_g'],m['vi']['pz']])
print('\n')

print('v x d h/dphig')
print(var['v'].reshape(len(ind))*dMh[im,2])
print(J[:,m['vi']['phi_g'],m['vi']['phi_gz']])
print('\n')

print('v x d h/dmh ')
print(var['v'].reshape(len(ind))*dMh[im,3])
print(J[:,m['vi']['phi_g'],m['vi']['mhz']])
print('\n')

print('Qh')
print(Qh[ind].T)
print(Q[:,m['vi']['phi_g']])
print('\n')
print('\n')



print('co2 continuity')
print('v x d c/dp ')
print(var['v'].reshape(len(ind))*dMc[im,0])
print( J[:,m['vi']['mh'],m['vi']['pz']])
print('\n')

print('v x d c/dphig')
print(var['v'].reshape(len(ind))*dMc[im,2])
print(J[:,m['vi']['mh'],m['vi']['phi_gz']])
print('\n')

print('v x d c/dmh')
print(var['v'].reshape(len(ind))*dMc[im,3])
print(J[:,m['vi']['mh'],m['vi']['mhz']])
print('\n')

print('Qc')
print(Qc[ind].T)
print(Q[:,m['vi']['mh']])
print('\n')
print('\n')


Check values (order: matlab; python)
solid phase fraction
[[0.39796074 0.40212897 0.40244963 0.40115024 0.40559331 0.49651157
  0.66533821]]
[[0.39796074 0.40212897 0.40244963 0.40115024 0.40559331 0.49651157
  0.66533821]]


liquid phase fraction
[[5.11099850e-01 4.75568630e-01 4.37747203e-01 3.96387588e-01
  3.46198620e-01 1.75226710e-01 5.54557090e-17]]
[[5.11099850e-01 4.75568630e-01 4.37747203e-01 3.96387588e-01
  3.46198620e-01 1.75226710e-01 5.54557090e-17]]


dissolved h2o
[[0.03855389 0.03584835 0.03284282 0.02964696 0.02630088 0.02198896
  0.00099875]]
[[0.03855389 0.03584835 0.03284282 0.02964696 0.02630088 0.02198896
  0.00099875]]


dissolved co2
[[4.29962943e-05 2.77509429e-05 1.86322475e-05 1.26283671e-05
  8.31381866e-06 4.11257847e-06 9.25111662e-09]]
[[4.29962943e-05 2.77509429e-05 1.86322475e-05 1.26283671e-05
  8.31381866e-06 4.11257847e-06 9.25111662e-09]]


viscosity
[[1.17687486e+05 1.93564957e+05 3.53338976e+05 7.77791121e+05
  2.96223545e+06 3.15140304e+11 7.52

In [47]:
# check that governing equations approximately equals to zero?

nu = np.concatenate((var['p'], var['v'], var['phi_g'], var['mh']), axis=1)
dnudt = np.concatenate((grd['p']['z'], grd['v']['z'], grd['phi_g']['z'], grd['mh']['z']), axis=1)
nu2 = np.concatenate((nu, dnudt), axis=1)

print(nu)
print(nu2.shape)

r = np.einsum('ijk,ik->ij',J,nu2) -Q

print(r)

[[1.05428000e+08 9.16509585e-04 7.04205832e-02 9.34573912e-01]
 [9.11278690e+07 9.46010577e-04 1.04605974e-01 9.51044146e-01]
 [7.73346180e+07 9.84791127e-04 1.44929390e-01 9.61217304e-01]
 [6.41450593e+07 1.03331969e-03 1.90346122e-01 9.68279922e-01]
 [5.16257654e+07 1.08956846e-03 2.38853760e-01 9.74018834e-01]
 [3.72333774e+07 1.18039723e-03 3.24321283e-01 9.82110351e-01]
 [1.00000028e+05 1.14392486e-03 3.34661794e-01 9.82110351e-01]]
(7, 8)
[[ 1.38112198e-10 -3.01837557e-15 -3.82560161e-07 -6.53848590e-08]
 [ 3.56931485e-13  1.76674350e-15 -1.21664924e-06 -1.52934392e-07]
 [ 8.47064118e-14 -7.37044946e-16 -3.04518593e-06 -3.00030827e-07]
 [-3.15396320e-12 -1.50296641e-15 -6.86269341e-06 -5.48992114e-07]
 [ 3.58320814e-12  4.18669065e-17 -1.45143579e-05 -9.45431463e-07]
 [ 7.57250165e-15 -3.68628739e-18 -1.75317240e-05 -7.79942768e-07]
 [-1.13463264e-14  4.33680869e-18 -1.90139213e-10 -8.45882107e-12]]


In [ ]:
eng.quit()